### Setup

In [1]:
#download the image data
%cd /content/
!gdown https://drive.google.com/uc?id=1HpFpsQ7VojeJbXj8i9i8IeIzXrzQ5zOs
!unzip "nerf_data.zip"; rm "nerf_data.zip"
# !gdown https://drive.google.com/uc?id=1ahp0Ac3I_7Fm1usqPT_Rvgww5KaLvo6Y
# !unzip davis.zip; rm davis.zip

/content
Downloading...
From: https://drive.google.com/uc?id=1HpFpsQ7VojeJbXj8i9i8IeIzXrzQ5zOs
To: /content/nerf_data.zip
113MB [00:01, 64.1MB/s]
Archive:  nerf_data.zip
   creating: nerf_data/
   creating: nerf_data/kid-running/
   creating: nerf_data/kid-running/dense/
   creating: nerf_data/kid-running/dense/images/
  inflating: nerf_data/kid-running/dense/images/00069.png  
  inflating: nerf_data/kid-running/dense/images/00065.png  
  inflating: nerf_data/kid-running/dense/images/00044.png  
  inflating: nerf_data/kid-running/dense/images/00062.png  
  inflating: nerf_data/kid-running/dense/images/00030.png  
  inflating: nerf_data/kid-running/dense/images/00002.png  
  inflating: nerf_data/kid-running/dense/images/00013.png  
  inflating: nerf_data/kid-running/dense/images/00036.png  
  inflating: nerf_data/kid-running/dense/images/00034.png  
  inflating: nerf_data/kid-running/dense/images/00009.png  
  inflating: nerf_data/kid-running/dense/images/00056.png  
  inflating: nerf_d

In [2]:
%cd /content
!git clone https://github.com/rohaldb/Neural-Scene-Flow-Fields
%cd /content/Neural-Scene-Flow-Fields
!pip install -r requirements.txt

/content
Cloning into 'Neural-Scene-Flow-Fields'...
remote: Enumerating objects: 615, done.
remote: Counting objects: 100% (322/322), done.
remote: Compressing objects: 100% (255/255), done.
remote: Total 615 (delta 197), reused 143 (delta 67), pack-reused 293
Receiving objects: 100% (615/615), 99.68 MiB | 23.39 MiB/s, done.
Resolving deltas: 100% (319/319), done.
/content/Neural-Scene-Flow-Fields
     |████████████████████████████████| 37.2MB 95kB/s 


In [3]:
%cd /content
!git clone https://github.com/rohaldb/multi-mono-sf.git
%cd multi-mono-sf
!chmod -R +x ./
!pip install -r requirements.txt

/content
Cloning into 'multi-mono-sf'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (133/133), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 133 (delta 49), reused 108 (delta 31), pack-reused 0
Receiving objects: 100% (133/133), 55.91 MiB | 21.16 MiB/s, done.
Resolving deltas: 100% (49/49), done.
/content/multi-mono-sf
     |████████████████████████████████| 61kB 6.3MB/s 
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [7]:
#download depth prediction model and move to appropriate place
!gdown https://drive.google.com/uc?id=1ROcvQzxtiBJOFY0MoLWk32emmzNOIAA4
!mv model.pt content/Neural-Scene-Flow-Fields/nsff_scripts

Downloading...
From: https://drive.google.com/uc?id=1ROcvQzxtiBJOFY0MoLWk32emmzNOIAA4
To: /content/Neural-Scene-Flow-Fields/nsff_scripts/model.pt
422MB [00:07, 53.1MB/s]
mv: cannot move 'model.pt' to 'content/Neural-Scene-Flow-Fields/nsff_scripts': No such file or directory


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Execution
First run the standard preprocessing pipeline of the neural-scene-flow paper. This will:
1. Convert the intrinsics/extrinsics to a special readable format
2. Resize the images and run a single view depth model to produce disparity estimates
3. Produce optical flow estimates
4. Use the optical flow estimates to obtain motion masks

We do not use the depth map or the optical flow for custom experiments. 

In [ ]:
%cd /content/Neural-Scene-Flow-Fields/nsff_scripts
# create camera intrinsics/extrinsic format for NSFF, same as original NeRF where it uses imgs2poses.py script from the LLFF code: https://github.com/Fyusion/LLFF/blob/master/imgs2poses.py
!python save_poses_nerf.py --data_path "/content/nerf_data/kid-running/dense/"
# Resize input images and run single view model
!python run_midas.py --data_path "/content/nerf_data/kid-running/dense/" --input_w 640 --input_h 360 --resize_height 288
# Run optical flow model (for easy setup and Pytorch version consistency, we use RAFT as backbond optical flow model, but should be easy to change to other models such as PWC-Net or FlowNet2.0)
!./download_models.sh
!python run_flows_video.py --model models/raft-things.pth --data_path /content/nerf_data/kid-running/dense/ --epi_threshold 1.0 --input_flow_w 768 --input_semantic_w 1024 --input_semantic_h 576

Next we run the mono-sf code, which will produce:
1. Disparity maps
2. Scene flow estimates
3. Induced optical flow estimates

By default, it will not produce visualisations to acompany the stored tensors. To include them in the output, add `--save_vis=True` to `eval_kid_running.sh`

In [5]:
# Run mono-sf to obtain scene flow, induced optical flow and disparity estimates
%cd /content/multi-mono-sf/scripts
!./eval_kid_running.sh

/content/multi-mono-sf/scripts
2021-07-06 04:16:18 ==> Commandline Arguments
2021-07-06 04:16:18   batch_size: 1
2021-07-06 04:16:18   batch_size_val: 1
2021-07-06 04:16:18   calculate_disparity_scale: False
2021-07-06 04:16:18   checkpoint: checkpoints/checkpoint_kitti_selfsup.ckpt
2021-07-06 04:16:18   checkpoint_exclude_params: ['']
2021-07-06 04:16:18   checkpoint_include_params: ['*']
2021-07-06 04:16:18   checkpoint_mode: resume_from_latest
2021-07-06 04:16:18   conv_padding_mode: zeros
2021-07-06 04:16:18   correlation_cuda_enabled: False
2021-07-06 04:16:18   cuda: True
2021-07-06 04:16:18   evaluation: True
2021-07-06 04:16:18   finetuning: False
2021-07-06 04:16:18   lr_scheduler: None
2021-07-06 04:16:18   model: MonoSceneFlow_Multi
2021-07-06 04:16:18   num_workers: 16
2021-07-06 04:16:18   optimizer: Adam
2021-07-06 04:16:18   optimizer_amsgrad: False
2021-07-06 04:16:18   optimizer_betas: (0.9, 0.999)
2021-07-06 04:16:18   optimizer_eps: 1e-08
2021-07-06 04:16:18   optimi

Next, remove the unneeded content (nsf scene flow & nsf disparity), zip the content and store in google drive.

In [7]:
# Remove irrelevant folders
# !rm -rf /content/nerf_data/kid-running/dense/flow_i1 #nsf optical flow
# !rm -rf /content/nerf_data/kid-running/dense/disp #nsf disparity

In [ ]:
#save to gdrive
%cd /content/
!zip -r custom_nerf_data.zip ./nerf_data
!cp -r custom_nerf_data.zip /content/gdrive/MyDrive/Thesis/Data
!ls /content/gdrive/MyDrive/Thesis/Data

/content
  adding: nerf_data/ (stored 0%)
  adding: nerf_data/kid-running/ (stored 0%)
  adding: nerf_data/kid-running/dense/ (stored 0%)
  adding: nerf_data/kid-running/dense/args.txt (deflated 52%)
  adding: nerf_data/kid-running/dense/poses_bounds.npy (deflated 19%)
  adding: nerf_data/kid-running/dense/disp_0/ (stored 0%)
  adding: nerf_data/kid-running/dense/disp_0/00012.npy (deflated 16%)
  adding: nerf_data/kid-running/dense/disp_0/00058.npy (deflated 17%)
  adding: nerf_data/kid-running/dense/disp_0/00037.npy (deflated 16%)
  adding: nerf_data/kid-running/dense/disp_0/00049.npy (deflated 16%)
  adding: nerf_data/kid-running/dense/disp_0/00031.npy (deflated 17%)
  adding: nerf_data/kid-running/dense/disp_0/00029.npy (deflated 17%)
  adding: nerf_data/kid-running/dense/disp_0/00030.npy (deflated 17%)
  adding: nerf_data/kid-running/dense/disp_0/00068.npy (deflated 18%)
  adding: nerf_data/kid-running/dense/disp_0/00035.npy (deflated 16%)
  adding: nerf_data/kid-running/dense/disp